In [2]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_ollama.chat_models import ChatOllama
from pydantic import BaseModel, Field
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

In [14]:
embed_model = OllamaEmbeddings(base_url="http://localhost:11434", model="bge-m3:latest")
print(embed_model)
vectorstore = Chroma(persist_directory="./db/chroma_langchain_db", embedding_function=embed_model)
retriever = vectorstore.as_retriever()

# retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={'k': 3, "fetch_k":5})

retriever

model='bge-m3:latest' base_url='http://localhost:11434' client_kwargs={}


VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001DEF060C560>, search_kwargs={})

In [15]:
# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

In [16]:
llm = ChatGroq(temperature=0, model_name= "llama-3.2-11b-text-preview")
# llm = ChatOllama(base_url="http://localhost:11434", model="llama3.2:latest")

structured_llm_grader = llm.with_structured_output(GradeDocuments)
structured_llm_grader

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001DEF060D7F0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001DEF060DEB0>, model_name='llama-3.2-11b-text-preview', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'GradeDocuments', 'description': 'Binary score for relevance check on retrieved documents.', 'parameters': {'properties': {'binary_score': {'description': "Documents are relevant to the question, 'yes' or 'no'", 'type': 'string'}}, 'required': ['binary_score'], 'type': 'object'}}}], 'tool_choice': {'type': 'function', 'function': {'name': 'GradeDocuments'}}}, config={}, config_factories=[])
| PydanticToolsParser(first_tool_only=True, tools=[<class '__main__.GradeDocuments'>])

In [17]:
# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)
retrieval_grader = grade_prompt | structured_llm_grader
retrieval_grader


ChatPromptTemplate(input_variables=['document', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="You are a grader assessing relevance of a retrieved document to a user question. \n \n    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n\n    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n\n    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['document', 'question'], input_types={}, partial_variables={}, template='Retrieved document: \n\n {document} \n\n User question: {question}'), additional_kwargs={})])
| RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001DEF060

In [19]:
%timeit
question = "what is noon report?"
docs = retriever.invoke(question)
docs

[]

In [73]:
doc_txt = docs[2].page_content
doc_txt

"pipe. 19)  Drawing of general layout showing plan / elevation / section view shall be included in. the approval drawing.. 20) Eye-plate to be suitably fitted for rotor of generaforsteam turbines, electric motors, pumpsheat exchanger's.cover/tube/bundles and.heawystrainer's.cover/filter/element of about 40 kg & above (And lifting eyes to have a min. diameter of 23 mm). 21) Test provision such as three way test cock to be provided for pressure switches and pressure transmitters. 22) The surfaces having normal operating temperatures above 60 C to be Insulated by temperature and for personal protection and at least the a preparation work(insulation stopper and/or pin) should be done by supplier before equipment delivery. AHYUNDAI HEAVY INDUSTRIES CO., LTD. PRINT 2024-06-17 07:38/H3#:/3037930"

In [74]:
result = retrieval_grader.invoke({"question": question, "document": doc_txt})
result.binary_score

'yes'

In [75]:
grade_no_docs = [grade_no_doc for grade_no_doc in docs if retrieval_grader.invoke({"question": question, "document": doc_txt}).binary_score == 'no']
grade_no_docs

[]

In [76]:
grade_yes_docs = [grade_yes_doc for grade_yes_doc in docs if retrieval_grader.invoke({"question": question, "document": doc_txt}).binary_score == 'yes']
grade_yes_docs

[Document(metadata={'File Name': 'FWG', 'File Path': '.\\documents\\pos\\FWG.pdf', 'First Division': 'documents', 'Page': 0, 'Second Division': 'pos'}, page_content='|None||None|None|None|F.W. GENERATOR|None|None|None|None|\n|TEL. NO.|SHIP NO.|None|None|None|None|None|None|None|None|\n|*1-6325|8250/8251|None|None|None|None|None|None|None|None|\n||APPROVED|None|Hyunseung Kim||None|None|None|None|None|\n|None||None||||None|None|None|None|\n|None|CHECKED|None|Seungho Han|||None|None|None|None|\n|None||None||||None|None|None|None|\n|None|DRAWN|None|Junsuk Yoon||REFERENCE DRAWING NO. 8250536-0|None|None|DRAWING SCALE|None|\n|HYUNDAI HEAVY INDUSTRIES CO., LTD. SAMHO SHIPYARD. KOREA|None|None|None|None|None|None|None|None|None|\n|None|None|None|None|None|SEQUENCE NO.|None||None|None|\n|None|None|None|None|None||None|None|None|None|'),
 Document(metadata={'File Name': 'FW Generator', 'File Path': '.\\documents\\pos\\FW Generator.pdf', 'First Division': 'documents', 'Page': 0, 'Second Division'